# Preprocessing

### Define Columns

In [107]:
import numpy as np
import pandas as pd

trees = pd.read_csv('../data/raw/new_york_tree_census_2015.csv')
# print(trees.columns)

target = trees['health']
keep_rows = target.notna()

drop_cols = ['health','tree_id', 'block_id', 'created_at', 'stump_diam','status','spc_common','problems','address',\
             'zip_city','cb_num', 'borocode', 'cncldist', 'st_assem', 'st_senate', 'nta', 'boro_ct', 'state',\
             'latitude', 'longitude']

group = ['nta_name']
spatial_geo = ['x_sp','y_sp'] # encode numerical, standardize
spatial_fine = ['zipcode'] # encode categorical
spatial_coarse = ['boroname'] # encode categorical

## base ftrs, without spatial ##
numerical_ftrs = ['tree_dbh']
categorical_ftrs = ['curb_loc', 'spc_latin', 'user_type', 'root_stone',
                    'root_grate', 'root_other', 'trunk_wire', 'trnk_light', 'trnk_other',
                    'brch_light', 'brch_shoe', 'brch_other']
ordinal_ftrs = ['steward','guards','sidewalk']
ordinal_cats = [['None','1or2','3or4','4orMore'],
                ['None','Harmful','Unsure','Helpful'],
                ['Damage','None','NoDamage']]

In [108]:
# check we have accounted for all columns
listed = set(drop_cols).union(group,spatial_geo,spatial_fine,spatial_coarse,\
                              numerical_ftrs,categorical_ftrs,ordinal_ftrs)
print(set(trees.columns).difference(listed) == set())

True


### Compose Transformers

In [109]:
# handle ordinal ftrs to fill in na
for ftr in ordinal_ftrs:
    trees[ftr] = trees[ftr].fillna('None')

In [110]:
from sklearn.model_selection import GroupKFold, GroupShuffleSplit, KFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.metrics import make_scorer, fbeta_score, accuracy_score
from sklearn.linear_model import LogisticRegression
## CROSS VALIDATION PIPELINE ##
RANDOM_STATE = 1
# define feature sets
drop_cols_set = drop_cols + group # + spatial_fine + spatial_coarse
categorical_ftrs_set = categorical_ftrs + spatial_fine + spatial_coarse
numerical_ftrs_set = numerical_ftrs + spatial_geo

# define a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('ord', OrdinalEncoder(categories=ordinal_cats), ordinal_ftrs),
        ('onehot', OneHotEncoder(sparse_output=False,handle_unknown='ignore'), categorical_ftrs_set),
        ('std', StandardScaler(), numerical_ftrs_set)])

Example pipeline:

```python
gkf = GroupKFold(n_splits=4,shuffle=True,random_state=RANDOM_STATE) 
gss = GroupShuffleSplit(n_splits=1,random_state=RANDOM_STATE)
# define algo
algo = LogisticRegression(penalty='elasticnet',solver='saga',max_iter=1000)
# make pipeline
pipe = make_pipeline(preprocessor,algo)
# define params
param_grid = {
            'logisticregression__C': [0.01, 0.1, 1, 10], 
            'logisticregression__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
            'logisticregression__random_state': [RANDOM_STATE]
            }
# make grid
grid = GridSearchCV(
    estimator=pipe, 
    param_grid=param_grid, 
    scoring='f1_macro',
    cv=gss,
    verbose=2,
    n_jobs=-1,
    refit=False
)
```

### Test Split and CV Subsample

In [111]:
# DATA SAMPLING #
RANDOM_STATE = 1
SAMPLE_PROP = 0.01
y = target[keep_rows]
X = trees[keep_rows].drop(drop_cols_set, axis=1)
groups = trees[keep_rows][group]
## hold out a test set by groups
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)
train_idx, test_idx = next(gss.split(X, y, groups))
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
groups_train, groups_test = groups.iloc[train_idx], groups.iloc[test_idx]
## sample 1% for cross validation
X_sub, _, y_sub, _, groups_sub, _ = train_test_split(
    X_train, y_train, groups_train,
    train_size=SAMPLE_PROP,
    stratify=y_train,
    random_state=RANDOM_STATE
)

# Cross Validation

### Logistic Regression with Elastic Net

In [112]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore", category=ConvergenceWarning)

In [113]:
random_states = [1,33,42,44,99]
results_scores = []
results_params = []
results_model = []
for rs in random_states:
    gss = GroupShuffleSplit(n_splits=1,random_state=rs)
    algo = LogisticRegression(penalty='elasticnet',solver='saga',max_iter=5_000)
    pipe = make_pipeline(preprocessor,algo)
    param_grid = {
            'logisticregression__C': [0.01, 0.1, 1, 10], 
            'logisticregression__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
            'logisticregression__class_weight': ['balanced'],
            'logisticregression__random_state': [rs]
            }
    grid = GridSearchCV(
        estimator=pipe, 
        param_grid=param_grid, 
        scoring='f1_macro',
        cv=gss,
        verbose=0,
        n_jobs=-1,
        refit=True,
        return_train_score=True
    )
    grid.fit(X_sub, y_sub, groups=groups_sub)
    results_params.append(grid.best_params_)
    best_idx = grid.best_index_
    results_scores.append(grid.cv_results_['mean_test_score'][best_idx])
    results_model.append(grid.best_estimator_)

In [ ]:
from json import dumps

print('best parameters:')
indx = np.argmax(results_scores)
print(dumps(results_params[indx],indent=4))
print(f'score: {results_scores[indx]}')

print(f'mean score: {np.mean(results_scores)}')
print(f'std score:  {np.std(results_scores)}')

# best parameters: 
# {
#     "logisticregression__C": 1,
#     "logisticregression__class_weight": "balanced",
#     "logisticregression__l1_ratio": 0.3,
#     "logisticregression__random_state": 42
# }
# score: 0.38616238364007627
# mean score: 0.37118256858651005
# std score:  0.011786660982739948

best parameters: 

{
    "logisticregression__C": 1,
    "logisticregression__class_weight": "balanced",
    "logisticregression__l1_ratio": 0.3,
    "logisticregression__random_state": 42
}
score: 0.38616238364007627

mean score: 0.37118256858651005
std score:  0.011786660982739948


### Logistic Regression with Elastic Net: `stratified`
We use `StratifiedShuffleSplit` instead of `GroupShuffleSplit`

In [115]:
# testing using random splitting instead of Group
from sklearn.model_selection import StratifiedShuffleSplit
random_states = [1,33,42,44,99]
results_scores = []
results_params = []
results_model = []
results_grid = []
for rs in random_states:
    gss = StratifiedShuffleSplit(n_splits=1,random_state=rs)
    algo = LogisticRegression(penalty='elasticnet',solver='saga',max_iter=10_000)
    pipe = make_pipeline(preprocessor,algo)
    param_grid = {
            'logisticregression__C': [0.01, 0.1, 1, 10], 
            'logisticregression__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
            'logisticregression__class_weight': ['balanced'],
            'logisticregression__random_state': [rs]
            }
    grid = GridSearchCV(
        estimator=pipe, 
        param_grid=param_grid, 
        scoring='f1_macro',
        cv=gss,
        verbose=0,
        n_jobs=-1,
        refit=True,
        return_train_score=True
    )
    grid.fit(X_sub, y_sub)
    results_params.append(grid.best_params_)
    best_idx = grid.best_index_
    results_scores.append(grid.cv_results_['mean_test_score'][best_idx])
    results_model.append(grid.best_estimator_)
    results_grid.append(grid)

In [ ]:
print('best parameters:')
indx = np.argmax(results_scores)
print(dumps(results_params[indx],indent=4))
print(f'score: {results_scores[indx]}')

print(f'mean score: {np.mean(results_scores)}')
print(f'std score:  {np.std(results_scores)}')

# best parameters:
# {
#     "logisticregression__C": 1,
#     "logisticregression__class_weight": "balanced",
#     "logisticregression__l1_ratio": 0.1,
#     "logisticregression__random_state": 33
# }
# score: 0.4142172190952678
# mean score: 0.39150945509422413
# std score:  0.0181171318594367

best parameters:
{
    "logisticregression__C": 1,
    "logisticregression__class_weight": "balanced",
    "logisticregression__l1_ratio": 0.1,
    "logisticregression__random_state": 33
}
score: 0.4142172190952678
mean score: 0.39150945509422413
std score:  0.0181171318594367


### XGBoost Classifier

Redo sampling (with the same random state), also producing an evaluation set for `early_stopping_rounds`

In [133]:
import xgboost as xgb
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.preprocessing import LabelEncoder
RANDOM_STATE = 1
# sample 5% for cv
SAMPLE_PROP = 0.05
# encode as numbers for xgboost 
y = target[keep_rows]
le = LabelEncoder()
y = pd.DataFrame(le.fit_transform(y))

X = trees[keep_rows].drop(drop_cols_set, axis=1)
groups = trees[keep_rows][group]
## hold out a test set by groups
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)
train_idx, test_idx = next(gss.split(X, y, groups))
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
groups_train, groups_test = groups.iloc[train_idx], groups.iloc[test_idx]
## sample for cross validation
X_sub, _, y_sub, _, groups_sub, _ = train_test_split(
    X_train, y_train, groups_train,
    train_size=SAMPLE_PROP,
    stratify=y_train,
    random_state=RANDOM_STATE
)
gss_eval = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)
sub_idx, eval_idx = next(gss_eval.split(X_sub, y_sub, groups_sub))
X_sub, X_eval = X.iloc[sub_idx], X.iloc[eval_idx]
y_sub, y_eval = y.iloc[sub_idx], y.iloc[eval_idx]
groups_sub, groups_eval = groups.iloc[sub_idx], groups.iloc[eval_idx]

# get weights
sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_sub
)

In [134]:
from copy import deepcopy
xg_pre = deepcopy(preprocessor)

xg_pre.fit(X)
X_sub_proc = xg_pre.transform(X_sub)
X_eval_proc = xg_pre.transform(X_eval)

In [151]:
# FIXME: remove n_estimators from grid and add early stopping 
# (validation set using 1% from 5% subsample

# testing using XGBoost
random_states = [1,33,42,44,99]
results_scores_xg = []
results_best_scores_xg = []
results_params_xg = []
results_model_xg = []
for rs in random_states:
    gss = GroupShuffleSplit(n_splits=1,random_state=rs)
    algo = xgb.XGBClassifier(early_stopping_rounds=10,verbosity=0)
    pipe = make_pipeline(algo)
    param_grid = {
            'xgbclassifier__learning_rate': [0.1],
            'xgbclassifier__max_depth': [3, 5, 7, 10],
            'xgbclassifier__n_jobs': [-1],
            'xgbclassifier__random_state': [rs],
            }
    grid = GridSearchCV(
        estimator=pipe, 
        param_grid=param_grid, 
        scoring='f1_macro',
        cv=gss,
        verbose=0,
        refit=True,
        return_train_score=True
    )
    grid.fit(X_sub_proc, 
             y_sub, 
             groups=groups_sub,
             xgbclassifier__sample_weight=sample_weights,
             xgbclassifier__eval_set=[(X_eval_proc,y_eval)])
    results_params_xg.append(grid.best_params_)
    results_scores_xg.append(grid.cv_results_)
    best_idx = grid.best_index_
    best_val_score = grid.cv_results_['mean_test_score'][best_idx]
    results_best_scores_xg.append(best_val_score)
    results_model_xg.append(grid.best_estimator_)

[0]	validation_0-mlogloss:1.09232
[1]	validation_0-mlogloss:1.08674
[2]	validation_0-mlogloss:1.08157
[3]	validation_0-mlogloss:1.07674
[4]	validation_0-mlogloss:1.07218
[5]	validation_0-mlogloss:1.06794
[6]	validation_0-mlogloss:1.06447
[7]	validation_0-mlogloss:1.06083
[8]	validation_0-mlogloss:1.05752
[9]	validation_0-mlogloss:1.05430
[10]	validation_0-mlogloss:1.05132
[11]	validation_0-mlogloss:1.04865
[12]	validation_0-mlogloss:1.04615
[13]	validation_0-mlogloss:1.04373
[14]	validation_0-mlogloss:1.04170
[15]	validation_0-mlogloss:1.03955
[16]	validation_0-mlogloss:1.03764
[17]	validation_0-mlogloss:1.03590
[18]	validation_0-mlogloss:1.03437
[19]	validation_0-mlogloss:1.03260
[20]	validation_0-mlogloss:1.03140
[21]	validation_0-mlogloss:1.02998
[22]	validation_0-mlogloss:1.02867
[23]	validation_0-mlogloss:1.02740
[24]	validation_0-mlogloss:1.02598
[25]	validation_0-mlogloss:1.02496
[26]	validation_0-mlogloss:1.02364
[27]	validation_0-mlogloss:1.02231
[28]	validation_0-mlogloss:1.0

In [150]:
print('best parameters:')
indx = np.argmax(results_best_scores_xg)
print(dumps(results_params_xg[indx],indent=4))
print(f'score: {results_best_scores_xg[indx]}')

print(f'mean score: {np.mean(results_best_scores_xg)}')
print(f'std score:  {np.std(results_best_scores_xg)}')

best_model = results_model_xg[indx]
print(f"best num trees: "
      f"{best_model.named_steps['xgbclassifier'].best_iteration}")

# best parameters:
# {
#     "xgbclassifier__learning_rate": 0.1,
#     "xgbclassifier__max_depth": 7,
#     "xgbclassifier__n_jobs": -1,
#     "xgbclassifier__random_state": 1
# }
# score: 0.3844493278296484
# mean score: 0.3824436024526744
# std score:  0.0020057253769740213
# best num trees: 99

best parameters:
{
    "xgbclassifier__learning_rate": 0.1,
    "xgbclassifier__max_depth": 7,
    "xgbclassifier__n_jobs": -1,
    "xgbclassifier__random_state": 1
}
score: 0.3844493278296484
mean score: 0.3824436024526744
std score:  0.0020057253769740213
best num trees: 99


In [ ]:
# testing using XGBoost
random_states = [1,99]
results_scores_xg = []
results_best_scores_xg = []
results_params_xg = []
results_model_xg = []
for rs in random_states:
    gss = StratifiedShuffleSplit(n_splits=1,random_state=rs)
    algo = xgb.XGBClassifier(early_stopping_rounds=10,verbosity=0)
    pipe = make_pipeline(algo)
    param_grid = {
            'xgbclassifier__learning_rate': [0.1],
            'xgbclassifier__max_depth': [3, 5, 7, 10],
            'xgbclassifier__n_jobs': [-1],
            'xgbclassifier__random_state': [rs],
            }
    grid = GridSearchCV(
        estimator=pipe, 
        param_grid=param_grid, 
        scoring='f1_macro',
        cv=gss,
        verbose=0,
        refit=True,
        return_train_score=True
    )
    grid.fit(X_sub_proc, 
             y_sub, 
             groups=groups_sub,
             xgbclassifier__sample_weight=sample_weights,
             xgbclassifier__eval_set=[(X_eval_proc,y_eval)])
    results_params_xg.append(grid.best_params_)
    results_scores_xg.append(grid.cv_results_)
    best_idx = grid.best_index_
    best_val_score = grid.cv_results_['mean_test_score'][best_idx]
    results_best_scores_xg.append(best_val_score)
    results_model_xg.append(grid.best_estimator_)

Fitting 1 folds for each of 9 candidates, totalling 9 fits
[CV] END xgbclassifier__learning_rate=0.1, xgbclassifier__max_depth=3, xgbclassifier__n_estimators=100, xgbclassifier__n_jobs=-1, xgbclassifier__random_state=1; total time=   0.6s
[CV] END xgbclassifier__learning_rate=0.1, xgbclassifier__max_depth=3, xgbclassifier__n_estimators=200, xgbclassifier__n_jobs=-1, xgbclassifier__random_state=1; total time=   1.1s
[CV] END xgbclassifier__learning_rate=0.1, xgbclassifier__max_depth=3, xgbclassifier__n_estimators=300, xgbclassifier__n_jobs=-1, xgbclassifier__random_state=1; total time=   1.6s
[CV] END xgbclassifier__learning_rate=0.1, xgbclassifier__max_depth=5, xgbclassifier__n_estimators=100, xgbclassifier__n_jobs=-1, xgbclassifier__random_state=1; total time=   0.8s
[CV] END xgbclassifier__learning_rate=0.1, xgbclassifier__max_depth=5, xgbclassifier__n_estimators=200, xgbclassifier__n_jobs=-1, xgbclassifier__random_state=1; total time=   1.5s
[CV] END xgbclassifier__learning_rate=0.1

In [46]:
print(dumps(results_params_xg_ss, indent=4))
print(results_best_scores_xg_ss)

# best params are
# max_depth = 7
# n_estimators = 300
# using Stratified split

[
    {
        "xgbclassifier__learning_rate": 0.1,
        "xgbclassifier__max_depth": 7,
        "xgbclassifier__n_estimators": 300,
        "xgbclassifier__n_jobs": -1,
        "xgbclassifier__random_state": 1
    },
    {
        "xgbclassifier__learning_rate": 0.1,
        "xgbclassifier__max_depth": 7,
        "xgbclassifier__n_estimators": 300,
        "xgbclassifier__n_jobs": -1,
        "xgbclassifier__random_state": 99
    }
]
[np.float64(0.42164081542427595), np.float64(0.4105136125596484)]


##### kneighbors

In [75]:
# using KNN Classifier
# sample #
SAMPLE_PROP = 0.01
y = target[keep_rows]
X = trees[keep_rows].drop(drop_cols_set, axis=1)
groups = trees[keep_rows][group]
## hold out a test set by groups
gss_test = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)
train_idx, test_idx = next(gss_test.split(X, y, groups))
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
groups_train, groups_test = groups.iloc[train_idx], groups.iloc[test_idx]
## sample 1% for cross validation
X_sub, _, y_sub, _, groups_sub, _ = train_test_split(
    X_train, y_train, groups_train,
    train_size=SAMPLE_PROP,
    stratify=y_train,
    random_state=RANDOM_STATE
)

In [71]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10, weights='distance', n_jobs=-1)
preprocessor.fit(X_train)
X_sub_proc = preprocessor.transform(X_sub)
X_test_proc = preprocessor.transform(X_test)
knn.fit(X_sub_proc, y_sub.to_numpy())

,n_neighbors,10
,weights,'distance'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,-1


In [72]:
preds = knn.predict(X_test_proc)
for val in np.unique_values(preds):
    print(f'{val}: {np.sum(preds==val)}')

Fair: 1644
Good: 118322
Poor: 98


In [73]:
print(fbeta_score(y_test.to_numpy(),preds,beta=1,average='macro'))

0.3138872856472891


In [76]:
# grid
random_states = [1,33,42,44,99]
results_scores_knn = []
results_best_scores_knn = []
results_params_knn = []
results_model_knn = []
for rs in random_states:
    gss = GroupShuffleSplit(n_splits=1,random_state=rs)
    algo = KNeighborsClassifier()
    pipe = make_pipeline(preprocessor,algo)
    param_grid = {
            'kneighborsclassifier__n_neighbors': [10,20,30,40,50],
            'kneighborsclassifier__weights': ['uniform','distance'],
            'kneighborsclassifier__n_jobs': [-1]
            }
    grid = GridSearchCV(
        estimator=pipe, 
        param_grid=param_grid, 
        scoring='f1_macro',
        cv=gss,
        verbose=0,
        refit=True,
        return_train_score=True
    )
    grid.fit(X_sub, 
             y_sub, 
             groups=groups_sub)
    results_params_knn.append(grid.best_params_)
    results_scores_knn.append(grid.cv_results_)
    best_idx = grid.best_index_
    best_val_score = grid.cv_results_['mean_test_score'][best_idx]
    results_best_scores_knn.append(best_val_score)
    results_model_knn.append(grid.best_estimator_)

In [78]:
indx = np.argmax(results_best_scores_knn)
print(results_best_scores_knn[indx])
print(results_params_knn[indx])

0.32166812107523574
{'kneighborsclassifier__n_jobs': -1, 'kneighborsclassifier__n_neighbors': 10, 'kneighborsclassifier__weights': 'distance'}


#### support vector machine classifier

In [89]:
from sklearn.svm import SVC
svc = SVC(kernel='linear')
preprocessor.fit(X_train)
X_sub_proc = preprocessor.transform(X_sub)
X_test_proc = preprocessor.transform(X_test)
svc.fit(X_sub_proc, y_sub.to_numpy())

,C,1.0
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [90]:
preds = svc.predict(X_test_proc)
for val in np.unique_values(preds):
    print(f'{val}: {np.sum(preds==val)}')

print(fbeta_score(y_test.to_numpy(),preds,beta=1,average='macro'))

Fair: 304
Good: 119662
Poor: 98
0.30241659305545165


In [102]:
from sklearn.utils.class_weight import compute_sample_weight
RANDOM_STATE = 1
# sample 5% for training
SAMPLE_PROP = 0.01
y = target[keep_rows]
X = trees[keep_rows].drop(drop_cols_set, axis=1)
groups = trees[keep_rows][group]
## hold out a test set by groups
gss_test = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)
train_idx, test_idx = next(gss.split(X, y, groups))
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
groups_train, groups_test = groups.iloc[train_idx], groups.iloc[test_idx]
## sample 10% for cross validation
X_sub, _, y_sub, _, groups_sub, _ = train_test_split(
    X_train, y_train, groups_train,
    train_size=SAMPLE_PROP,
    stratify=y_train,
    random_state=RANDOM_STATE
)

# get weights
class_weights = {
    'Poor': np.sum(y.to_numpy()=='Poor') / len(y),
    'Fair': np.sum(y.to_numpy()=='Fair') / len(y),
    'Good': np.sum(y.to_numpy()=='Good') / len(y)
}
# sample_weights = compute_sample_weight(
#     class_weight='balanced',
#     y=y_sub
# )

In [105]:
# grid
random_states = [1,33,42,44,99]
results_scores_svc = []
results_best_scores_svc = []
results_params_svc = []
results_model_svc = []
for rs in random_states:
    gss = GroupShuffleSplit(n_splits=1,random_state=rs)
    algo = SVC(class_weight=class_weights)
    pipe = make_pipeline(preprocessor,algo)
    param_grid = {
            'svc__C': [0.01, 0.1, 1, 10],
            'svc__kernel': ['linear','poly','rbf','sigmoid']
            }
    grid = GridSearchCV(
        estimator=pipe, 
        param_grid=param_grid, 
        scoring='f1_macro',
        cv=gss,
        verbose=0,
        refit=True,
        n_jobs=-1,
        return_train_score=True
    )
    grid.fit(X_sub, 
             y_sub, 
             groups=groups_sub
             )
    results_params_svc.append(grid.best_params_)
    results_scores_svc.append(grid.cv_results_)
    best_idx = grid.best_index_
    best_val_score = grid.cv_results_['mean_test_score'][best_idx]
    results_best_scores_svc.append(best_val_score)
    results_model_svc.append(grid.best_estimator_)

In [106]:
indx = np.argmax(results_best_scores_svc)
print(results_best_scores_svc[indx])
print(results_params_svc[indx])

0.3198561828532294
{'svc__C': 10, 'svc__kernel': 'sigmoid'}


##### Try `xgboost` models on test data. 
```python 
[TODO] ... 
```

In [ ]:
# train a bigger model

xg_model = results_model_xg[0]
test_score_xg = fbeta_score(y_test, xg_model.predict(X_test), beta=1, average='macro')
print(f'xg group split: {test_score_xg}')


xg group split: 0.39001143285292533
xg stratify split: 0.39001143285292533


In [ ]:
xg_ss_model = results_model_xg_ss[0]
test_score_xg_ss = fbeta_score(y_test, xg_ss_model.predict(X_test), beta=1, average='macro')
print(f'xg stratify split: {test_score_xg_ss}')

#### train our final model on 50% and evaluate it for shap

In [ ]:
## CROSS VALIDATION PIPELINE ##
RANDOM_STATE = 1
# define feature sets
drop_cols_set = drop_cols + group
categorical_ftrs_set = categorical_ftrs + spatial_fine + spatial_coarse
numerical_ftrs_set = numerical_ftrs + spatial_geo

# define a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('ord', OrdinalEncoder(categories=ordinal_cats), ordinal_ftrs),
        ('onehot', OneHotEncoder(sparse_output=False,handle_unknown='ignore'), categorical_ftrs_set),
        ('std', StandardScaler(), numerical_ftrs_set)])

clf = xgb.XGBClassifier(n_estimators=300,
                            learning_rate=0.1,
                            max_depth=7,
                            n_jobs=-1,
                            random_state=RANDOM_STATE,
                            verbosity=2)

pipeline = make_pipeline(preprocessor,clf)

In [76]:
SAMPLE_PROP = 0.5
# encode as numbers for xgboost 
y = target[keep_rows]
le = LabelEncoder()
y = pd.DataFrame(le.fit_transform(y))

X = trees[keep_rows].drop(drop_cols_set, axis=1)
groups = trees[keep_rows][group]

## hold out a test set by groups
gss_test = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)
train_idx, test_idx = next(gss.split(X, y, groups))
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
groups_train, groups_test = groups.iloc[train_idx], groups.iloc[test_idx]
## sample 10% for cross validation
# X_sub, _, y_sub, _, groups_sub, _ = train_test_split(
#     X_train, y_train, groups_train,
#     train_size=SAMPLE_PROP,
#     stratify=y_train,
#     random_state=RANDOM_STATE
# )

# get weights
sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_train
)

In [77]:
pipeline.fit(X_train, y_train, xgbclassifier__sample_weight=sample_weights)

[16:01:27] INFO: /Users/runner/miniforge3/conda-bld/xgboost-split_1764148547996/work/src/data/iterative_dmatrix.cc:56: Finished constructing the `IterativeDMatrix`: (532108, 351, 186769908).


,steps,"[('columntransformer', ...), ('xgbclassifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('ord', ...), ('onehot', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [88]:
# Good = 1,
# Poor = 2,
# Fair = 0
print(dict(zip(le.classes_, range(len(le.classes_)))))

{'Fair': 0, 'Good': 1, 'Poor': 2}


In [80]:
final_score = fbeta_score(y_test, pipeline.predict(X_test), beta=1, average='macro')
print(final_score)

0.3866518605085765


In [89]:
baseline_preds = y_test.copy()
baseline_preds.iloc[:, 0] = 1
baseline_score = fbeta_score(y_test, baseline_preds, beta=1, average='macro')
print(baseline_score)

0.2990431574273948


##### shap explainer

In [118]:
import shap
preprocessor = ColumnTransformer(
    transformers=[
        ('ord', OrdinalEncoder(categories=ordinal_cats), ordinal_ftrs),
        ('onehot', OneHotEncoder(sparse_output=False,handle_unknown='ignore'), categorical_ftrs_set),
        ('std', StandardScaler(), numerical_ftrs_set)])

clf = xgb.XGBClassifier(n_estimators=300,
                            learning_rate=0.1,
                            max_depth=7,
                            n_jobs=-1,
                            random_state=RANDOM_STATE,
                            verbosity=2)

y = target[keep_rows]
le = LabelEncoder()
y = pd.DataFrame(le.fit_transform(y))

X = trees[keep_rows].drop(drop_cols_set, axis=1)
groups = trees[keep_rows][group]

## hold out a test set by groups
gss_test = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)
train_idx, test_idx = next(gss.split(X, y, groups))
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]

X_train_prep = preprocessor.fit_transform(X_train)
feature_names = preprocessor.get_feature_names_out()
df_train = pd.DataFrame(data=X_train_prep,columns=feature_names)
X_test_prep = preprocessor.transform(X_test)
df_test = pd.DataFrame(data=X_test_prep,columns = feature_names)


y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
groups_train, groups_test = groups.iloc[train_idx], groups.iloc[test_idx]

sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_train
)

clf.fit(df_train, y_train, sample_weight=sample_weights)


[16:36:49] INFO: /Users/runner/miniforge3/conda-bld/xgboost-split_1764148547996/work/src/data/iterative_dmatrix.cc:56: Finished constructing the `IterativeDMatrix`: (532108, 351, 186769908).


,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [119]:
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X_test_prep, check_additivity=False)

ValueError: could not convert string to float: '[3.3342963E-1,3.329155E-1,3.3328208E-1]'